# Prueba Técnica – Analista 3  
### Servicios de Nómina a Empleados  
**Shamuel Steven Molina Duque**

---

## Introducción

En esta prueba técnica me compartieron varios archivos en formato Excel con información relevante para el proceso de nómina. Para su análisis y procesamiento, opté por utilizar **DuckDB**, un motor de base de datos SQL embebido y optimizado para análisis de datos directamente desde archivos como CSV, Excel o `DataFrames` de Python, sin necesidad de levantar un servidor.  
Es comparable a **SQLite**, pero diseñado específicamente para tareas analíticas.

Aprovechando la naturaleza liviana del dataset y el número limitado de tablas, decidí mantener una solución simple, pero estructurada, utilizando este notebook en Python como entorno principal de trabajo.


In [40]:
# Librerias:
import pandas as pd
import duckdb
import numpy as np

In [41]:
df1 = pd.read_excel('bonificaciones.xlsx')
df2 = pd.read_excel('deducciones.xlsx')
df3 = pd.read_excel('empleados.xlsx')
df4 = pd.read_excel('horas_extras.xlsx')

# Regitro de los dataframes en Duckdb
con = duckdb.connect()

# Registrarlos como tablas temporales para limpiar y generar la tabla de nomina
# que será el resultado de unirlas todas (luego de limpiar un poco)
con.register('bonificaciones', df1)
con.register('deducciones', df2)
con.register('empleados', df3)
con.register('horas_extras', df4)

### La Query que ves a continuación es la que limpia y genera la tabla de "nomia" a la cual le aplicaremos los cálculos

Esta Query es compatible con Impala, asi que si se ejecuta en la LZ, obtendrás el mismo resultado. Tambien la adjunté como el archivo .sql en el correo de la entrega.

In [42]:
con.execute("""
    CREATE TABLE nomina AS
    WITH horas_extras_limpia AS (
        SELECT DISTINCT id_empleado, fecha, tipo_hora, cantidad_horas
        FROM horas_extras
        WHERE id_empleado IS NOT NULL AND fecha IS NOT NULL
    ),
    empleados_limpia AS (
        SELECT DISTINCT id_empleado, fecha_ingreso, nombre, tipo_contrato, salario_basico
        FROM empleados
        WHERE id_empleado IS NOT NULL
    ),
    deducciones_limpia AS (
        SELECT DISTINCT id_empleado, concepto, valor
        FROM deducciones
        WHERE id_empleado IS NOT NULL AND concepto IS NOT NULL
    ),
    bonificaciones_limpia AS (
        SELECT DISTINCT id_empleado, concepto, valor
        FROM bonificaciones
        WHERE id_empleado IS NOT NULL AND concepto IS NOT NULL
    )
    SELECT 
        e.id_empleado,
        e.nombre,
        e.fecha_ingreso,
        e.tipo_contrato,
        e.salario_basico,
        he.fecha AS fecha_hora_extra,
        he.tipo_hora,
        he.cantidad_horas,
        d.concepto AS concepto_deduccion,
        d.valor AS valor_deduccion,
        b.concepto AS concepto_bonificacion,
        b.valor AS valor_bonificacion
    FROM empleados_limpia e
    LEFT JOIN horas_extras_limpia he ON e.id_empleado = he.id_empleado
    LEFT JOIN deducciones_limpia d ON e.id_empleado = d.id_empleado
    LEFT JOIN bonificaciones_limpia b ON e.id_empleado = b.id_empleado
""")

In [43]:
nomina = con.execute("SELECT * FROM nomina").fetchdf()

In [44]:
nomina.head()

,id_empleado,nombre,fecha_ingreso,tipo_contrato,salario_basico,fecha_hora_extra,tipo_hora,cantidad_horas,concepto_deduccion,valor_deduccion,concepto_bonificacion,valor_bonificacion
0,5444,Empleado_4443,2024-03-16 00:00:00,Fijo,5320695,2024-06-10 00:00:00,nocturna,2,préstamo_nomina,702297,cumplimiento,103617
1,3578,Empleado_2577,2022-04-02 00:00:00,Indefinido,7107402,2024-06-24 00:00:00,diurna,2,préstamo_nomina,444795,extraordinaria,524054
2,1514,Empleado_513,2023-02-14 00:00:00,Fijo,7520599,2024-06-15 00:00:00,diurna,2,retencion_fuente,751177,extraordinaria,163560
3,5968,Empleado_4967,2024-01-26 00:00:00,Obra Labor,7589780,2024-06-12 00:00:00,diurna,1,embargo_judicial,907624,cumplimiento,277610
4,3259,Empleado_2258,2021-12-15 00:00:00,Obra Labor,8579032,2024-06-11 00:00:00,diurna,4,retencion_fuente,549270,extraordinaria,583243


### Integración de datos empleando las reglas de negocio:

In [45]:
# Asegurarnos que las columnas fecha_ingreso y fecha_hora_extra están como datetime
nomina['fecha_ingreso'] = pd.to_datetime(nomina['fecha_ingreso'], errors='coerce')
nomina['fecha_hora_extra'] = pd.to_datetime(nomina['fecha_hora_extra'], errors='coerce')

# Filtrar solo empleados activos antes de junio 2024
nomina_activos = nomina[nomina['fecha_ingreso'] < '2024-06-01'].copy()

# Filtrar solo registros de horas extra de junio 2024
nomina_activos = nomina_activos[nomina_activos['fecha_hora_extra'].dt.to_period('M') == '2024-06']

# Calcular valor hora ordinaria 
nomina_activos['valor_hora'] = nomina_activos['salario_basico'] / 240

# Calcular valor total de la hora extra según tipo
def calcular_valor_hora_extra(row):
    if row['tipo_hora'] == 'diurna':
        return row['cantidad_horas'] * row['valor_hora'] * 1.25
    elif row['tipo_hora'] == 'nocturna':
        return row['cantidad_horas'] * row['valor_hora'] * 1.75
    else:
        return 0

nomina_activos['valor_horas_extras'] = nomina_activos.apply(calcular_valor_hora_extra, axis=1)

resumen = nomina_activos.groupby(['id_empleado', 'nombre', 'salario_basico'], as_index=False).agg({
    'valor_horas_extras': 'sum',
    'valor_bonificacion': 'sum',
    'valor_deduccion': 'sum'
})

resumen.rename(columns={
    'valor_horas_extras': 'total_horas_extras',
    'valor_bonificacion': 'total_bonificaciones',
    'valor_deduccion': 'total_deducciones'
}, inplace=True)

# CÁLCULO DEL SALARIO LIQUIDO
resumen['salario_liquido'] = (
    resumen['salario_basico'] +
    resumen['total_horas_extras'] +
    resumen['total_bonificaciones'] -
    resumen['total_deducciones']
)

# Calcular salario bruto (sin deducciones) para alerta por bonificación
resumen['salario_bruto'] = (
    resumen['salario_basico'] +
    resumen['total_horas_extras'] +
    resumen['total_bonificaciones']
)

# Generar columna de alerta financiera
# Esta columna nos da valores 0 y 1, siendo:
# Alerta financiera = 1
# Sin alerta financiera = 0
# Tambien se le agrega un sufijo "Bon" o "Dedu" según la alerta sea por bonificación o deducción
def alerta_financiera(row):
    if row['salario_bruto'] > 10_000_000:
        return '1_Bon'
    elif row['total_deducciones'] > 700_000:
        return '1_Dedu'
    else:
        return '0'

resumen['alerta_financiera'] = resumen.apply(alerta_financiera, axis=1)

# (Opcional) eliminar columna auxiliar si no la necesitas
resumen.drop(columns='salario_bruto', inplace=True)

In [46]:
resumen.head(-1)

,id_empleado,nombre,salario_basico,total_horas_extras,total_bonificaciones,total_deducciones,salario_liquido,alerta_financiera
0,1001,Empleado_0,2771908,57748.083333,1610919,1222683,3217892.083333,1_Dedu
1,1003,Empleado_2,7080048,752255.100000,5664056,7818660,5677699.1,1_Bon
2,1004,Empleado_3,4113232,89976.950000,1046083,2959161,2290130.95,1_Dedu
3,1005,Empleado_4,9419945,98124.427083,0,1951256,7566813.427083,1_Dedu
4,1006,Empleado_5,2339250,24367.187500,693648,0,3057265.1875,0
...,...,...,...,...,...,...,...,...
3887,5994,Empleado_4993,9495183,98908.156250,262067,0,9856158.15625,0
3888,5995,Empleado_4994,9081889,378412.041667,789870,0,10250171.041667,1_Bon
3889,5996,Empleado_4995,5994586,268507.497917,480802,1233860,5510035.497917,1_Dedu
3890,5997,Empleado_4996,8034978,502186.125000,2901948,0,11439112.125,1_Bon


### Finalmente exportamos como un excel para alimentar el tablero de PowerBi

In [47]:
resumen.to_excel("Shamuel Molina Duque - Excel Resultado.xlsx", index=False)

Ya a modo de resumen, en este ejercicio se integraron y procesaron datos provenientes de una tabla consolidada de nómina formada a partir de las 4 tablas que se me suministraron para el ejercicio, que incluía información de horas extras, bonificaciones, deducciones y datos básicos de empleados. A partir de esta, se calcularon los valores correspondientes al salario líquido mensual para junio de 2024, considerando el pago diferenciado por tipo de hora extra y las reglas de negocio establecidas. Además, se implementó una validación financiera para alertar casos en los que el salario bruto superaba los 10 millones tras bonificaciones o cuando las deducciones excedían los \$700.000, generando un reporte final estructurado y exportable.
